In [0]:
import pandas as pd

In [0]:
data1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/imdb review model/word2vec-nlp-tutorial/labeledTrainData.tsv',delimiter="\t")

In [0]:
data2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/imdb review model/word2vec-nlp-tutorial/imdb_master.csv',encoding='latin-1')

In [0]:
data1= data1.drop('id',axis=1)

In [0]:
data2.drop(['Unnamed: 0','type','file'],axis=1,inplace=True)

In [0]:
data2.columns = ['review','sentiment']

In [0]:
data2 = data2[data2['sentiment'] != 'unsup']

In [0]:
data2['sentiment'] = data2['sentiment'].map({'pos':1,'neg':0})

In [0]:
data = pd.concat([data1,data2]).reset_index(drop=True)

In [10]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


In [0]:
def clean_text(text):
  text = re.sub(r'[^\w\s]','',text,re.UNICODE)
  text = text.lower()
  text = nltk.word_tokenize(text)
  text = [lemmatizer.lemmatize(token,'v') for token in text]
  text = [word for word in text if not word in stop_words]
  text = ' '.join(text)
  return text

In [13]:
data.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


In [0]:
data['Cleaned_data'] = data['review'].apply(lambda x: clean_text(x))

In [15]:
data.head()

,sentiment,review,Cleaned_data
0,1,With all this stuff going down at the moment w...,stuff go moment mj ive start listen music watc...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi...",classic war worlds timothy hines entertain fil...
2,0,The film starts with a manager (Nicholas Bell)...,film start manager nicholas bell give welcome ...
3,0,It must be assumed that those who praised this...,must assume praise film greatest film opera ev...
4,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy wondrously unpretentious 80s e...


In [16]:
data['Cleaned_data'].apply(lambda x: len(x.split(" "))).mean()

146.23453333333333

In [0]:
from keras.preprocessing.text import Tokenizer

In [0]:
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data['Cleaned_data'])
list_tokenized_train = tokenizer.texts_to_sequences(data['Cleaned_data'])

In [0]:
from keras.preprocessing.sequence import pad_sequences
# convert list of sequnces to a 2D numpy array of shape (num_samples,num_timestamps)

max_len = 130 # is num_timesteps. if this max_len not given. then largest sequence is taken
# max_len is  number of columns in this array

# sequences shorter than num_timestamps(130 here), will be padded till they are 130 in len.
X_t = pad_sequences(list_tokenized_train,maxlen=max_len)
y = data['sentiment']

In [42]:
X_t.shape

(75000, 130)

In [0]:
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
# Embedding layer is initialized with random weights and will learn ....
# ...an embedding for all of the words in the training dataset

embed_size = 128 #  size of the vector space in which words will be embedded. test for best valu!! 
# embed_size is the dimensions(columns) of each vector. where every word is a vector.
model = Sequential()  
model.add(Embedding(max_features,embed_size))
model.add(Bidirectional(LSTM(32,return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20,activation = 'tanh'))
model.add(Dropout(0.05))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [65]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 128)         768000    
_________________________________________________________________
bidirectional_6 (Bidirection (None, None, 64)          41216     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 20)                1300      
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 21        
Total params: 810,537
Trainable params: 810,537
Non-trainable params: 0
________________________________________________

In [66]:
batch_size = 100
epochs = 3
model.fit(X_t,y,batch_size=batch_size,epochs=epochs,validation_split=0.2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 60000 samples, validate on 15000 samples
Epoch 1/3
60000/60000 [==============================] - 152s 3ms/step - loss: 0.3189 - accuracy: 0.8562 - val_loss: 0.2436 - val_accuracy: 0.9030
Epoch 2/3
60000/60000 [==============================] - 147s 2ms/step - loss: 0.2047 - accuracy: 0.9203 - val_loss: 0.2051 - val_accuracy: 0.9279
Epoch 3/3
60000/60000 [==============================] - 146s 2ms/step - loss: 0.1529 - accuracy: 0.9434 - val_loss: 0.1591 - val_accuracy: 0.9449


In [0]:
model.save('movie_model.h5')

In [0]:
data_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/imdb review model/word2vec-nlp-tutorial/testData.tsv',delimiter="\t")

In [80]:
data_test.head()

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [0]:
data_test['Cleaned_data'] = data_test['review'].apply(lambda x: clean_text(x))

In [86]:
data_test.head()

,id,review,Cleaned_data
0,12311_10,Naturally in a film who's main themes are of m...,naturally film whos main theme mortality nosta...
1,8348_2,This movie is a disaster within a disaster fil...,movie disaster within disaster film full great...
2,5828_4,"All in all, this is a movie for kids. We saw i...",movie kid saw tonight child love one point kid...
3,7186_2,Afraid of the Dark left me with the impression...,afraid dark leave impression several different...
4,12128_7,A very accurate depiction of small time mob li...,accurate depiction small time mob life film ne...


In [0]:
data_test["sentiment"] = data_test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)

In [89]:
data_test.head()

,id,review,Cleaned_data,sentiment
0,12311_10,Naturally in a film who's main themes are of m...,naturally film whos main theme mortality nosta...,1
1,8348_2,This movie is a disaster within a disaster fil...,movie disaster within disaster film full great...,0
2,5828_4,"All in all, this is a movie for kids. We saw i...",movie kid saw tonight child love one point kid...,0
3,7186_2,Afraid of the Dark left me with the impression...,afraid dark leave impression several different...,0
4,12128_7,A very accurate depiction of small time mob li...,accurate depiction small time mob life film ne...,1


In [0]:
data_test.drop('review',axis=1,inplace=True)

In [93]:
data_test.columns=['id','review','sentiment']
data_test.head()

,id,review,sentiment
0,12311_10,naturally film whos main theme mortality nosta...,1
1,8348_2,movie disaster within disaster film full great...,0
2,5828_4,movie kid saw tonight child love one point kid...,0
3,7186_2,afraid dark leave impression several different...,0
4,12128_7,accurate depiction small time mob life film ne...,1


In [0]:
y_test = data_test["sentiment"]
list_sentence_test = data_test['review']
list_tokenize_test = tokenizer.texts_to_sequences(list_sentence_test)

In [0]:
X_test = pad_sequences(list_tokenize_test,maxlen=max_len)
prediction=model.predict(X_test)
y_pred=prediction>0.5

In [102]:
from sklearn.metrics import f1_score,confusion_matrix
f1_score(y_test,y_pred)

0.9639402646808855

In [103]:
confusion_matrix(y_test,y_pred)

array([[12200,   300],
       [  591, 11909]])